In [ ]:
import pandas as pd
import requests
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark
import os
from datetime import datetime
import json

In [ ]:
now = datetime.now()
update_dt = now.strftime("%Y-%m-%d %H:%M:%S")
today = now.strftime("%Y-%m-%d")

In [ ]:
ZABBIX_API_URL = "http://192.168.10.109/zabbix/api_jsonrpc.php"
UNAME = "Admin"
PWORD = "zabbix"

In [ ]:
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['HADOOP_USER_NAME']='hive'
os.environ['PYSPARK_PYTHON'] ='/root/anaconda3/bin/python'
conf = pyspark.SparkConf().setAll([
     ('spark.driver.maxResultSize', '0'),
     ('spark.driver.memory', '2g'),
     ('spark.sql.repl.eagerEval.enabled','true'),
     ("spark.driver.allowMultipleContexts", "true"),
     ('hive.strict.managed.tables','false'),
     ('hive.metastore.uris', 'thrift://hive-metastore:9083'),
     ('metastore.client.capability.check','false')
    ])
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("Zabbix") \
        .config(conf=conf) \
        .enableHiveSupport() \
        .getOrCreate()

In [ ]:
def Auth():
    r = requests.post(ZABBIX_API_URL,
                  json={
                      "jsonrpc": "2.0",
                      "method": "user.login",
                      "params": {
                          "user": UNAME,
                          "password": PWORD},
                      "id": 1
                  })
    AUTHTOKEN = r.json()["result"]
    return AUTHTOKEN

In [ ]:
def get_item():
    r = requests.post(ZABBIX_API_URL, json={
        "jsonrpc": "2.0",
        "method": "item.get",
        "params": {
            "output": "extend",
        },
            "auth": Auth(),
            "id": 1
    })
    data = []
    for row in r.json()['result']:
        data.append(row)
    return data

In [ ]:
readfile = pd.DataFrame(get_item())

In [ ]:
readfile = readfile.replace("\r\n",'', regex=True)
readfile = readfile.replace("\[]",'', regex=True)

In [ ]:
readfile.to_csv('/tmp/get_item.csv', index=False, header=True)

In [ ]:
df = spark.read.csv('/tmp/get_item.csv', header=True)

In [ ]:
if spark.sql("SHOW TABLES IN main").filter("tableName == 'zbx_item'").count() > 0:
    # df.write \
    # .mode("append") \
    # .partitionBy("hostid") \
    # .saveAsTable("main.zbx_item")
    print("Table is Exists")
else:
    df.write \
    .mode("overwrite") \
    .partitionBy("hostid") \
    .saveAsTable("main.zbx_item")